In [ ]:
debugging = True
IPTS = '19558'

# Description 

# Python Import

In [ ]:
from __code.__all import custom_style
custom_style.style()

In [ ]:
%gui qt

In [ ]:
from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_linear_profile.ui')

In [ ]:
from __code.normalization import NormalizationHandler

from __code import utilities
working_dir = utilities.get_working_dir(ipts=IPTS, debugging=debugging)
print("Working dir: {}".format(working_dir))

# Select Sample Images

In [ ]:
o_norm_handler = NormalizationHandler(working_dir = working_dir)
o_norm_handler.select_images()

# Load Data

In [ ]:
o_norm_handler.load_data()

# Select Profile

In [ ]:
from __code.ui_linear_profile import Ui_MainWindow as UiMainWindow

import pyqtgraph as pg
from pyqtgraph.dockarea import *

try:
    from PyQt4.QtGui import QFileDialog
    from PyQt4 import QtCore, QtGui
    from PyQt4.QtGui import QMainWindow
except ImportError:
    from PyQt5.QtWidgets import QFileDialog
    from PyQt5 import QtCore, QtGui
    from PyQt5.QtWidgets import QApplication, QMainWindow

In [ ]:
import numpy as np
import copy
from types import SimpleNamespace


class LinearProfile(QMainWindow):
    
    table_profile_header = ['name', 'x0', 'y0','x1', 'y1', 'width', 'color']
    index_column = {'name': 0,
                    'x0': 1,
                    'y0': 2,
                    'x1': 3,
                    'y1': 4,
                    'width': 5,
                    'color': 6}
    live_data = None
    
    def __init__(self, parent=None, list_files=[], data=[]):

        QMainWindow.__init__(self, parent=parent)
        self.ui = UiMainWindow()
        self.ui.setupUi(self)
        self.setWindowTitle("Linear Profile")
        
        # disable table_profile
        self.ui.table_profile.blockSignals(True)
        
        self.data = data
        self.list_files = list_files
        self.nbr_files = len(list_files)
        
        self.init_pyqtgraph()
        self.init_widgets()
        
        self.__refresh_plot()
        
        self.__roi = {'name': '',
                      'x0': 50,
                      'y0': 100,
                      'x1': -200,
                      'y1': -300,
                      'width': 1,
                      'color': None}
        self.list_rois = []
        self.list_rois_ui = []
        self.ui.table_profile.blockSignals(False)
            
    # initialization
    def init_widgets(self):
        self.ui.file_index_slider.setMaximum(self.nbr_files-1)
        
    def init_pyqtgraph(self):
        area = DockArea()
        area.setVisible(True)
        d1 = Dock("Sample Image", size=(200, 300))
        d2 = Dock("Profile", size=(200, 100))
        
        area.addDock(d1, 'top')
        area.addDock(d2, 'bottom')
        
        # image view
        self.ui.image_view = pg.ImageView()
        self.ui.image_view.ui.menuBtn.hide()
        self.ui.image_view.ui.roiBtn.hide()
        d1.addWidget(self.ui.image_view)
        
        # profile
        self.profile = pg.PlotWidget(title='Profile')
        self.profile.plot()
        d2.addWidget(self.profile)
        
        # set up layout
        vertical_layout = QtGui.QVBoxLayout()
        vertical_layout.addWidget(area)
        self.ui.widget.setLayout(vertical_layout)
    
    # widgets signals
    def slider_changed(self, index=0):
        self.__refresh_plot(index=index)
        
    def add_profile(self):
        nbr_roi = len(self.list_rois)
        new_roi = copy.deepcopy(self.__roi)
        new_roi['name'] = "region_{}".format(nbr_roi)
        self.list_rois.append(new_roi)
        self.__update_roi_table()
        
        x0 = new_roi['x0']
        y0 = new_roi['y0']
        x1 = new_roi['x1']
        y1 = new_roi['y1']
        width = new_roi['width']
        color = new_roi['color']
        
        _roi = pg.LineROI([x0,y0], [x1,y1], width=1, pen=(6,9))  # [[0, x1], [y0, y1]]
        _roi.sigRegionChanged.connect(self.graphical_rois_changed)
        self.list_rois_ui.append(_roi)
        self.ui.image_view.addItem(_roi)
        
    def graphical_rois_changed(self):
        for _index, _roi_ui in enumerate(self.list_rois_ui):
            region = _roi_ui.getArraySlice(self.live_data,
                                          self.ui.image_view.imageItem)
            x0 = region[0][0].start
            x1 = region[0][0].stop
            y0 = region[0][1].start
            y1 = region[0][1].stop
            
            roi = self.list_rois[_index]
            roi['x0'] = x0
            roi['x1'] = x1
            roi['y0'] = y0
            roi['y1'] = y1
            self.list_rois[_index] = roi
        
        self.__update_roi_table()
        
    def manual_rois_changed(self):
        """table of rois has been modified"""
        nbr_row = self.ui.table_profile.rowCount()
        
        self.list_rois = []
        for _row_index in np.arange(nbr_row):

            _roi = copy.deepcopy(self.__roi)

            _roi['name']= self._get_item_value(row=_row_index, column=0)
            _roi['x0'] = self._get_item_value(row=_row_index, column=1)
            _roi['y0'] = self._get_item_value(row=_row_index, column=2)
            _roi['x1'] = self._get_item_value(row=_row_index, column=3)
            _roi['y1'] = self._get_item_value(row=_row_index, column=4)
            _roi['width'] = self._get_item_value(row=_row_index, column=5)
        
            x0 = np.int(_roi['x0'])
            y0 = np.int(_roi['y0'])
            x1 = np.int(_roi['x1'])
            y1 = np.int(_roi['y1'])
            width = np.int(_roi['width'])
     
            self.list_rois.append(_roi)
            
            _roi = pg.LineROI([x0,y0], [x1,y1], width=width, pen=(6,9))  # [[0, x1], [y0, y1]]
            _roi.sigRegionChanged.connect(self.graphical_rois_changed)
            self.ui.image_view.removeItem(self.list_rois_ui[_row_index])
            self.ui.image_view.addItem(_roi)
            self.list_rois_ui[_row_index] = _roi
        
    def _get_item_value(self, row=0, column=0):
        return str(self.ui.table_profile.item(row, column).text())
        
    def clear_table_profile(self):
        nbr_row = self.ui.table_profile.rowCount()
        for _ in np.arange(nbr_row):
            self.ui.table_profile.removeRow(0)
        
    def __update_roi_table(self):
        self.ui.table_profile.blockSignals(True)
        self.clear_table_profile()
        for _row, _roi_dict in enumerate(self.list_rois):
            self.ui.table_profile.insertRow(_row)
            for _column, _key in enumerate(_roi_dict):
                self.insert_item(row=_row,
                                column=_column,
                                value=_roi_dict[_key])
        self.ui.table_profile.blockSignals(False)
    
    def save_roi_table(self, row, column):
        if (row == -1) and (column == -1): # full save
            new_list_roi = []
            nbr_row = self.ui.table_profile.rowCount()
            nbr_column = self.ui.table_profile.columnCount()
            for _row in np.arange(nbr_row):
#                _roi = self.list_roi[row]
                _new_roi = copy.deepcopy(self.__roi)
                for _column in np.arange(nbr_column):
                    _new_roi[self.table_profile_header[_column]] = str(self.ui.table_profile.item(_row, _column).text())
                new_list_roi.append(_new_roi)
            self.list_rois = new_list_roi
        else:
            _roi = self.list_rois[row]
            _roi[self.table_profile_header[column]] = str(self.ui.table_profile.item(row, column).text())
            self.list_rois[row] = _roi
            
        self.manual_rois_changed()
        
    
    def insert_item(self, row=0, column=0, value=''):
        _item = QtGui.QTableWidgetItem(str(value))
        self.ui.table_profile.setItem(row, column, _item)
    
    def remove_profile(self):
        _row_selected = self.ui.table_profile.currentRow()
        if _row_selected == -1:
            return
        roi_ui_removed = self.list_rois_ui.pop(_row_selected)
        self.ui.image_view.removeItem(roi_ui_removed)
        self.ui.table_profile.removeRow(_row_selected)
        self.save_roi_table(-1, -1)
        
    def ok_button_clicked(self):
        _roi_ui = self.list_rois_ui[0]
        self.my_roi = _roi_ui
        region = _roi_ui.getArraySlice(self.live_data,
                                      self.ui.image_view.imageItem)
        
        # this is to get the counts profile
#         region = _roi_ui.getArrayRegion(self.live_data,
#                                         self.ui.image_view.imageItem,
#                                        axes=(0,1))
        print(region)
        print("yo")
        
    def export_button_clicked(self):
        print("export button clicked")

    # private methods
    def __refresh_plot(self, index=np.NaN):
        if np.isnan(index):
            index = self.ui.file_index_slider.value()
        _data = self.data[index]
        self.live_data = _data
        self.ui.image_view.setImage(_data)


In [ ]:
o_linear_profile = LinearProfile(list_files= o_norm_handler.list_file_names['sample'],
                                 data=o_norm_handler.data['sample'])
o_linear_profile.show()

In [ ]:
o_linear_profile.my_roi.getArraySlice(o_linear_profile.live_data, o_linear_profile.ui.image_view.imageItem)